# 디렉토리 설정 및 이미지데이터 전처리

In [28]:
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
%cd drive/MyDrive/CNN_upgrade/

[Errno 2] No such file or directory: 'drive/MyDrive/CNN_upgrade/'
/content/drive/MyDrive/CNN_upgrade


In [31]:
!pwd

/content/drive/My Drive/CNN_upgrade


In [32]:
base_dir = './'
categories = ['datasets/pedestrian', 'datasets/sitter', 'datasets/taxier']
nb_classes = len(categories)

In [33]:
# 이미지 크기 지정
image_w = 256
image_h = 256
pixels = image_w*image_h*3

In [34]:
# 이미지데이터 읽어들이기
X = []
Y = []
for idx, category in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    # 이미지
    image_dir = base_dir+category+'/*'
    files = glob.glob(image_dir)
    
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
    
X = np.array(X)
Y = np.array(Y)

# 학습 전용 데이터와 테스트 전용 데이터로 구분
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [35]:
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (674, 256, 256, 3)
X_test shape: (225, 256, 256, 3)


# 일반 CNN 모델 적용

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), (1, 1), input_shape=X_train.shape[1:], padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# 전결합층
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 64)       

In [37]:
# 모델 구축하기
from tensorflow.keras import metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.CategoricalAccuracy()])

In [38]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=10)
mc = ModelCheckpoint(base_dir+'CNN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
histroy = model.fit(X_train, y_train, batch_size=16, epochs=100, callbacks=[es, mc], validation_split=0.15)

Epoch 1/100
36/36 [==============================] - 52s 1s/step - loss: 1.4994 - categorical_accuracy: 0.3110 - val_loss: 1.0970 - val_categorical_accuracy: 0.4020
Epoch 2/100
36/36 [==============================] - 53s 1s/step - loss: 1.0970 - categorical_accuracy: 0.3741 - val_loss: 1.0956 - val_categorical_accuracy: 0.4020
Epoch 3/100
36/36 [==============================] - 52s 1s/step - loss: 1.0946 - categorical_accuracy: 0.3666 - val_loss: 1.0818 - val_categorical_accuracy: 0.3922
Epoch 4/100
36/36 [==============================] - 51s 1s/step - loss: 1.0565 - categorical_accuracy: 0.4428 - val_loss: 0.9940 - val_categorical_accuracy: 0.5000
Epoch 5/100
36/36 [==============================] - 51s 1s/step - loss: 0.8765 - categorical_accuracy: 0.5837 - val_loss: 0.6223 - val_categorical_accuracy: 0.7647
Epoch 6/100
36/36 [==============================] - 51s 1s/step - loss: 0.6877 - categorical_accuracy: 0.7007 - val_loss: 0.5125 - val_categorical_accuracy: 0.8039
Epoch 7/10

In [39]:
# 모델 평가하기 
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

8/8 [==============================] - 5s 629ms/step - loss: 0.4357 - categorical_accuracy: 0.8800
loss= 0.43570443987846375
accuracy= 0.8799999952316284


In [40]:
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')

for category in categories:
    for i in range(1,4):
        test_image = base_dir+category+'_test_'+str(i)+'.jpg'
        
        # 이미지 resize
        img = Image.open(test_image)
        img = img.convert("RGB")
        img = img.resize((256,256))
        data = np.asarray(img)
        X = np.array(data)
        X = X.astype("float") / 256
        X = X.reshape(-1, 256,256,3)
        
        # 예측
        pred = model.predict(X)
        print(pred)
        result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
        print(test_image, ':',categories[result[0]])
        print()

[[9.9998236e-01 5.1498118e-06 1.2561673e-05]]
./datasets/pedestrian_test_1.jpg : datasets/pedestrian

[[9.9973923e-01 1.1610368e-04 1.4463370e-04]]
./datasets/pedestrian_test_2.jpg : datasets/pedestrian

[[9.9975246e-01 1.3242716e-05 2.3436367e-04]]
./datasets/pedestrian_test_3.jpg : datasets/pedestrian

[[9.9965763e-01 2.2325113e-04 1.1919963e-04]]
./datasets/sitter_test_1.jpg : datasets/pedestrian

[[1.2614774e-02 9.8732275e-01 6.2499719e-05]]
./datasets/sitter_test_2.jpg : datasets/sitter

[[5.3759669e-03 9.9461669e-01 7.3083547e-06]]
./datasets/sitter_test_3.jpg : datasets/sitter

[[2.9367834e-04 6.0376027e-05 9.9964595e-01]]
./datasets/taxier_test_1.jpg : datasets/taxier

[[1.7859059e-04 2.8891118e-05 9.9979252e-01]]
./datasets/taxier_test_2.jpg : datasets/taxier

[[7.6607705e-07 6.6261907e-07 9.9999857e-01]]
./datasets/taxier_test_3.jpg : datasets/taxier

